Importando Paquetes

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
import nltk, re, time
from nltk.corpus import stopwords
from collections import defaultdict
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from collections import namedtuple

Using TensorFlow backend.


Importando el Dataset

In [3]:
#--------------------Extracting Emotion && sentences from corpus
def extract(dataset):
    f = open(dataset, 'r+')
    linea = f.readline()
    emotion = []
    sentences = []
    neutro = re.compile('^ne')
    while linea != "":    
        #Ignoramos Neutro emotions
        if not re.match(neutro, linea):
            #print("** ",linea)
            linea = linea.split("#")
            #Obtaining the emocion
            emotion.append(int(linea[0]))
            sentences.append(linea[1])
        linea = f.readline()
    f.close()
    return sentences,emotion

dataset = "Heterogenious_Dataset/dataset.txt"
sentences,emotion = extract(dataset)
print(emotion[0]," ",sentences[0])
print("Labels: ",len(emotion), "Sentences: ",len(sentences))

2   Mortar assault leaves at least 18 dead

Labels:  3695 Sentences:  3695


In [5]:
def clean_text(text, remove_stopwords=True):
    '''Clean the text, with the option to remove stopwords'''
    
    # Convert words to lower case and split them
    text = text.lower().split()

    # Optionally, remove stop words
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        text = [w for w in text if not w in stops]
    
    text = " ".join(text)

    # Clean the text
    text = re.sub(r"<br />", " ", text)
    text = re.sub(r"[^a-z]", " ", text)
    text = re.sub(r"   ", " ", text) # Remove any extra spaces
    text = re.sub(r"  ", " ", text)
    
    # Return a list of words
    return(text)

In [9]:
data_clean = []
for sentence in sentences:
    data_clean.append(clean_text(sentence))


In [10]:
# Inspect the cleaned reviews
for i in range(3):
    print(data_clean[i])
    print()

mortar assault leaves least dead

goal delight sheva

nigeria hostage feared dead freed



In [11]:
# Tokenize the reviews
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data_clean)
print("Fitting is complete.")

data_seq = tokenizer.texts_to_sequences(data_clean)
print("data_seq is complete.")


Fitting is complete.
data_seq is complete.


In [12]:
# Find the number of unique tokens
word_index = tokenizer.word_index
print("Words in index: %d" % len(word_index))

Words in index: 7705


In [13]:
# Inspect the reviews after they have been tokenized
for i in range(3):
    print(data_seq[i])
    print()

[3513, 2298, 413, 497, 112]

[1358, 1108, 3514]

[1109, 3515, 1681, 112, 1110]



In [14]:
# Find the length of reviews
lengths = []
for sentence in data_seq:
    lengths.append(len(sentence))

# Create a dataframe so that the values can be inspected
lengths = pd.DataFrame(lengths, columns=['counts'])

In [15]:
lengths.counts.describe()

count    3695.000000
mean        8.333965
std         5.971913
min         1.000000
25%         5.000000
50%         6.000000
75%        11.000000
max        51.000000
Name: counts, dtype: float64

In [30]:
print(np.percentile(lengths.counts, 80))
print(np.percentile(lengths.counts, 85))
print(np.percentile(lengths.counts, 90))
print(np.percentile(lengths.counts, 95))

12.0
14.0
16.0
20.0


I'm going to use 200 as the maximum length of a review. Although longer would be better, I want to limit the training time, and this value will still provide the full text to more than 80% of the reviews.

In [31]:
# Pad and truncate the questions so that they all have the same length.
max_review_length = 20

data_pad = pad_sequences(data_seq, maxlen = max_review_length)
print("data_pad is complete.")


data_pad is complete.


In [32]:
# Inspect the reviews after padding has been completed. 
for i in range(3):
    print(data_pad[i])
    print()

[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0 3513 2298  413  497  112]

[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0 1358 1108 3514]

[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0 1109 3515 1681  112 1110]



In [33]:
print(len(emotion))

3695


In [129]:
# Creating the training and validation sets
#x_train, x_test, y_train, y_test = train_test_split(data_pad, emotion,test_size = 0.15, random_state = 2)
#x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train,test_size = 0.15, random_state = 2)
x_train, x_valid, y_train, y_valid = train_test_split(data_pad, emotion,test_size = 0.15, random_state = 2)

In [131]:
# Inspect the shape of the data
print(x_train.shape)
print(x_valid.shape)
#print(x_test.shape)

(3140, 20)
(555, 20)


In [132]:
def toLabels(emo):
    classes = 6
    Y = np.zeros((len(emo), classes)) #Num Ejemplos X 6 emociones
    for e in range(0,len(emo)):
        vector = np.zeros((classes))
        vector[(emo[e] % (classes + 1)) - 1] = 1 
        Y[e] = vector
        #print(emo[e],"---",vector)
    return Y
 
y_train = toLabels(y_train)
#y_test = toLabels(y_test)
y_valid = toLabels(y_valid)

In [133]:
print("Y_train: ",y_train.shape," Num Ejem X # clases")
print("X_train: ",x_train.shape," Num Ejem X Maximun Lengh")
#print("Y_test: ",y_test.shape," Num Ejem X # clases")
#print("X_test: ",x_test.shape," Num Ejem X Maximun Lengh")
print("Y_valid: ",y_valid.shape," Num Ejem X # clases")
print("X_valid: ",x_valid.shape," Num Ejem X Maximun Lengh")

Y_train:  (3140, 6)  Num Ejem X # clases
X_train:  (3140, 20)  Num Ejem X Maximun Lengh
Y_valid:  (555, 6)  Num Ejem X # clases
X_valid:  (555, 20)  Num Ejem X Maximun Lengh


Build and Train the Model

In [38]:
def get_batches(x, y, batch_size):
    '''Create the batches for the training and validation data'''
    n_batches = len(x)//batch_size
    x, y = x[:n_batches*batch_size], y[:n_batches*batch_size]
    for ii in range(0, len(x), batch_size):
        yield x[ii:ii+batch_size], y[ii:ii+batch_size]

In [39]:
def get_test_batches(x, batch_size):
    '''Create the batches for the testing data'''
    n_batches = len(x)//batch_size
    x = x[:n_batches*batch_size]
    for ii in range(0, len(x), batch_size):
        yield x[ii:ii+batch_size]

In [205]:
def build_rnn(n_words, embed_size, batch_size, lstm_size, num_layers, 
              dropout, learning_rate, multiple_fc, fc_units):
    '''Build the Recurrent Neural Network'''

    tf.reset_default_graph()

    # Declare placeholders we'll feed into the graph
    with tf.name_scope('inputs'):
        inputs = tf.placeholder(tf.int32, [None, None], name='inputs')

    with tf.name_scope('labels'):
        labels = tf.placeholder(tf.int32, [None, None], name='labels')

    keep_prob = tf.placeholder(tf.float32, name='keep_prob')

    # Create the embeddings
    with tf.name_scope("embeddings"):
        embedding = tf.Variable(tf.random_uniform((n_words, embed_size), -1, 1))
        embed = tf.nn.embedding_lookup(embedding, inputs)
        
        #data = tf.Variable(tf.zeros([batchSize, maxSeqLength, numDimensions]),dtype=tf.float32)
        #data = tf.nn.embedding_lookup(wordVectors,input_data)

    # Build the RNN layers
    with tf.name_scope("RNN_layers"):
        lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size)
        drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
        cell = tf.contrib.rnn.MultiRNNCell([drop] * num_layers)
    
    # Set the initial state
    with tf.name_scope("RNN_init_state"):
        initial_state = cell.zero_state(batch_size, tf.float32)

    # Run the data through the RNN layers
    with tf.name_scope("RNN_forward"):
        outputs, final_state = tf.nn.dynamic_rnn(cell, embed,
                                                 initial_state=initial_state)    
    
    # Create the fully connected layers
    with tf.name_scope("fully_connected"):
        
        # Initialize the weights and biases
        weights = tf.truncated_normal_initializer(stddev=0.1)
        biases = tf.zeros_initializer()
        
        dense = tf.contrib.layers.fully_connected(outputs[:, -1],
                                                  num_outputs = fc_units,
                                                  activation_fn = tf.sigmoid,
                                                  weights_initializer = weights,
                                                  biases_initializer = biases)
        dense = tf.contrib.layers.dropout(dense, keep_prob)
        
        # Depending on the iteration, use a second fully connected layer
        if multiple_fc == True:
            dense = tf.contrib.layers.fully_connected(dense,
                                                      num_outputs = fc_units,
                                                      activation_fn = tf.sigmoid,
                                                      weights_initializer = weights,
                                                      biases_initializer = biases)
            dense = tf.contrib.layers.dropout(dense, keep_prob)
    
    # Make the predictions
    with tf.name_scope('predictions'):
        predictions = tf.contrib.layers.fully_connected(dense, 
                                                        num_outputs = 6, 
                                                        activation_fn=tf.sigmoid,
                                                        weights_initializer = weights,
                                                        biases_initializer = biases)
        tf.summary.histogram('predictions', predictions)
    
    # Calculate the cost
    with tf.name_scope('cost'):
        cost = tf.losses.mean_squared_error(labels, predictions)
        tf.summary.scalar('cost', cost)
    
    # Train the model
    with tf.name_scope('train'):    
        optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

    # Determine the accuracy
    with tf.name_scope("accuracy"):
        correct_pred = tf.equal(tf.cast(tf.round(predictions), tf.int32), labels)
        accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
        tf.summary.scalar('accuracy', accuracy)
   
    # Merge all of the summaries
    merged = tf.summary.merge_all()    

    # Export the nodes 
    export_nodes = ['inputs', 'labels', 'keep_prob', 'initial_state', 'final_state','accuracy',
                    'predictions', 'cost', 'optimizer', 'merged']
    Graph = namedtuple('Graph', export_nodes)
    local_dict = locals()
    graph = Graph(*[local_dict[each] for each in export_nodes])
    
    return graph

In [223]:
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score

def train(model, epochs, log_string):
    '''Train the RNN'''

    saver = tf.train.Saver()
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())

        # Used to determine when to stop the training early
        valid_loss_summary = []
        
        # Keep track of which batch iteration is being trained
        iteration = 0

        print()
        print("Training Model: {}".format(log_string))

        train_writer = tf.summary.FileWriter('./logs/3/train/{}'.format(log_string), sess.graph)
        valid_writer = tf.summary.FileWriter('./logs/3/valid/{}'.format(log_string))

        for e in range(epochs):
            state = sess.run(model.initial_state)
            
            # Record progress with each epoch
            train_loss = []
            train_acc = []
            val_acc = []
            val_loss = []
            
            #
            val_pre = []
            val_recall = []
            val_f1 = []

            with tqdm(total=len(x_train)) as pbar:
                #total_batch = int(2669/batch_size)
                #X_batches = np.array_split(x_train, batch_size)
                #Y_batches = np.array_split(y_train, batch_size)
                
                #for i in range(total_batch):
                 #   x, y = X_batches[i], Y_batches[i]     
                for _, (x, y) in enumerate(get_batches(x_train, y_train, batch_size), 1):
                    feed = {model.inputs: x,
                            model.labels: y,
                            model.keep_prob: dropout,
                            model.initial_state: state}
                    summary, loss, acc, state, _ = sess.run([model.merged, 
                                                             model.cost, 
                                                             model.accuracy, 
                                                             model.final_state, 
                                                             model.optimizer], 
                                                            feed_dict=feed)                
                    
                    # Record the loss and accuracy of each training batch
                    train_loss.append(loss)
                    train_acc.append(acc)
                    
                    # Record the progress of training
                    train_writer.add_summary(summary, iteration)
                    
                    iteration += 1
                    pbar.update(batch_size)
            
            # Average the training loss and accuracy of each epoch
            avg_train_loss = np.mean(train_loss)
            avg_train_acc = np.mean(train_acc) 

            val_state = sess.run(model.initial_state)
            with tqdm(total=len(x_valid)) as pbar:
                #total_batch = int(471/batch_size)
                #X_batches = np.array_split(x_valid, batch_size)
                #Y_batches = np.array_split(y_valid, batch_size)
      
                #for i in range(total_batch):
                 #   x, y = X_batches[i], Y_batches[i] 
                for x, y in get_batches(x_valid, y_valid, batch_size):
                    feed = {model.inputs: x,
                            model.labels: y,
                            model.keep_prob: 1,
                            model.initial_state: val_state}
                    summary, batch_loss, batch_acc, val_state, batch_pre = sess.run([model.merged, 
                                                                          model.cost, 
                                                                          model.accuracy, 
                                                                          model.final_state,
                                                                          model.predictions], 
                                                                         feed_dict=feed)
                    
                    # Record the validation loss and accuracy of each epoch
                    val_loss.append(batch_loss)
                    val_acc.append(batch_acc)
                    pbar.update(batch_size)
                    
                    result = tf.argmax(batch_pre , 1).eval()
                    test = (tf.argmax(y_valid , 1).eval()) 
                    y_true = test[:55]
                    y_pred = result
                    precision = precision_score(y_true, y_pred, average=None)
                    recall = recall_score(y_true, y_pred, average=None)
                    f1 = f1_score(y_true, y_pred, average=None)
                    val_pre.append(precision)
                    val_recall.append(recall)
                    val_f1.append(f1)
                    

                    #print("Predicc tensor: ",batch_pre.shape) #Preddciones del Batch
                    #print("Labels: ",y_valid.shape)
                    
                    
            
            # Average the validation loss and accuracy of each epoch
            avg_valid_loss = np.mean(val_loss)    
            avg_valid_acc = np.mean(val_acc)
            avg_pre = np.mean(val_pre)
            avg_recall = np.mean(val_recall)
            avg_f1 = np.mean(val_f1)
            valid_loss_summary.append(avg_valid_loss)
            
            #for i in range(0,55):
             #           print(y_true[i]," ",y_pred[i])
            
            # Record the validation data's progress
            valid_writer.add_summary(summary, iteration)

            # Print the progress of each epoch
            print("Epoch: {}/{}".format(e, epochs),
                  "Train Loss: {:.3f}".format(avg_train_loss),
                  "Train Acc: {:.3f}".format(avg_train_acc),
                  "Valid Loss: {:.3f}".format(avg_valid_loss),
                  "Valid Acc: {:.3f}".format(avg_valid_acc),
                  "Valid Pre: {:.3f}".format(avg_pre),
                  "Valid Reca: {:.3f}".format(avg_recall),
                  "Valid F1: {:.3f}".format(avg_f1))

            ########################### Start eport ##########################
            '''result = tf.argmax(batch_pre , 1).eval({model.inputs: x, model.labels: y})
            test = (tf.argmax(y_valid , 1).eval()) #Convierte de [0,0,1,0,0] a [2] y lo hace np.array

            #print("Test: ",test.shape)
            #print("Result: ",result.shape)
                    
            y_true = test[:55]
            y_pred = result
            #print(confusion_matrix(y_true, y_pred))
            class_names = ["Hapinnes","Sadness","Anger","Disust","Surprise","Fear"]
            from sklearn.metrics import classification_report
            print(classification_report(y_true, y_pred, target_names=class_names))
            '''
            ########################### End Report ##########################

            
            
            #checkpoint = "C:/Users/danie/Desktop/Checkpoints/sentiment_{}.ckpt".format(log_string)
            #saver.save(sess, checkpoint)
            # Stop training if the validation loss does not decrease after 3 epochs
            if avg_valid_loss > min(valid_loss_summary):
                print("No Improvement.")
                #stop_early += 1
                #if stop_early == 3:
                #    break   
            
            # Reset stop_early if the validation loss finds a new low
            # Save a checkpoint of the model
            else:
                print("New Record!")
                stop_early = 0
                checkpoint = "C:/Users/danie/Desktop/Checkpoints/sentiment_{}.ckpt".format(log_string)
                saver.save(sess, checkpoint)

In [230]:
# The default parameters of the model
n_words = len(word_index)
embed_size = 300
batch_size = 55 #250
lstm_size = 128
num_layers = 1
dropout = 0.5
learning_rate = 0.01
epochs = 1000
multiple_fc = False
fc_units = 256

In [231]:
# Train the model with the desired tuning parameters
for lstm_size in [64,128]:
    for multiple_fc in [True, False]:
        for fc_units in [128, 256]:
            log_string = 'ru={},fcl={},fcu={}'.format(lstm_size,
                                                      multiple_fc,
                                                      fc_units)
model = build_rnn(n_words = n_words, 
                              embed_size = embed_size,
                              batch_size = batch_size,
                              lstm_size = lstm_size,
                              num_layers = num_layers,
                              dropout = dropout,
                              learning_rate = learning_rate,
                              multiple_fc = multiple_fc,
                              fc_units = fc_units)            
train(model, epochs, log_string)


Training Model: ru=128,fcl=False,fcu=256


  0%|          | 0/555 [00:00<?, ?it/s]C:\Users\danie\Anaconda3\envs\Compositor\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\danie\Anaconda3\envs\Compositor\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
 99%|█████████▉| 550/555 [00:01<00:00, 485.04it/s]


Epoch: 0/1000 Train Loss: 0.146 Train Acc: 0.822 Valid Loss: 0.129 Valid Acc: 0.841 Valid Pre: 0.089 Valid Reca: 0.173 Valid F1: 0.091
New Record!


 99%|█████████▉| 550/555 [00:01<00:00, 478.96it/s]


Epoch: 1/1000 Train Loss: 0.115 Train Acc: 0.860 Valid Loss: 0.127 Valid Acc: 0.848 Valid Pre: 0.123 Valid Reca: 0.174 Valid F1: 0.132
New Record!


...

Epoch: 708/1000 Train Loss: 0.014 Train Acc: 0.985 Valid Loss: 0.198 Valid Acc: 0.797 Valid Pre: 0.196 Valid Reca: 0.211 Valid F1: 0.189
No Improvement.


 99%|█████████▉| 550/555 [03:09<00:01,  2.72it/s]


Epoch: 709/1000 Train Loss: 0.016 Train Acc: 0.983 Valid Loss: 0.194 Valid Acc: 0.803 Valid Pre: 0.206 Valid Reca: 0.219 Valid F1: 0.199
No Improvement.


 40%|███▉      | 220/555 [01:03<01:25,  3.93it/s]


KeyboardInterrupt: 

Predicciones

In [82]:
def make_predictions(lstm_size, multiple_fc, fc_units, checkpoint):
    '''Predict the sentiment of the testing data'''
    
    # Record all of the predictions
    all_preds = []

    model = build_rnn(n_words = n_words, 
                      embed_size = embed_size,
                      batch_size = batch_size,
                      lstm_size = lstm_size,
                      num_layers = num_layers,
                      dropout = dropout,
                      learning_rate = learning_rate,
                      multiple_fc = multiple_fc,
                      fc_units = fc_units) 
    
    with tf.Session() as sess:
        saver = tf.train.Saver()
        # Load the model
        saver.restore(sess, checkpoint)
        test_state = sess.run(model.initial_state)
        for _, x in enumerate(get_test_batches(x_valid, batch_size), 1):
            feed = {model.inputs: x,
                    model.keep_prob: 1,
                    model.initial_state: test_state}
            predictions = sess.run(model.predictions, feed_dict=feed)
            for pred in predictions:
                all_preds.append((pred))
                
    return all_preds

In [83]:
checkpoint1 = "C:/Users/danie/Desktop/Checkpoints/sentiment_ru=128,fcl=False,fcu=256.ckpt"
# Make predictions using the best 3 models
predictions1 = make_predictions(128, False, 256, checkpoint1)

INFO:tensorflow:Restoring parameters from C:/Users/danie/Desktop/Checkpoints/sentiment_ru=128,fcl=False,fcu=256.ckpt


In [86]:
def write_submission(predictions, string):
    '''write the predictions to a csv file'''
    submission = pd.DataFrame(data={"id":1, "sentiment":predictions})
    submission.to_csv("submission_{}.csv".format(string), index=False, quoting=3)

In [87]:
write_submission(predictions1, "ru=128,fcl=False,fcu=256") 

In [151]:
print(x_valid[149][19])
x_valid[149][19] = 281   

7705
